In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required packages (run this in Kaggle notebook cell)
!pip install pydicom opencv-python tqdm timm segmentation-models-pytorch

# Imports
import os
import numpy as np
import pydicom
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, jaccard_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import warnings
from scipy.spatial.distance import directed_hausdorff
from scipy import ndimage
import timm
import torch.nn.functional as F
import segmentation_models_pytorch as smp
warnings.filterwarnings('ignore')

# Kaggle-specific configurations
print("🚀 Starting Femur Segmentation Evaluation with Fixed ViT Model")
print("=" * 60)

# Device setup
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️  Using CPU (consider enabling GPU in Kaggle)")

# Paths - Updated for Kaggle environment
raw_path = '/kaggle/input/unet-dataset/data/raw'
mask_path = '/kaggle/input/unet-dataset/data/mask'

# Alternative paths for different dataset structures
if not os.path.exists(raw_path):
    possible_paths = [
        '/kaggle/input/*/raw',
        '/kaggle/input/*/images',
        '/kaggle/input/*/train/images',
        'data/raw',
        '../input/*/raw'
    ]
    
    print("🔍 Dataset not found at default path. Searching...")
    for path_pattern in possible_paths:
        import glob
        matches = glob.glob(path_pattern)
        if matches:
            raw_path = matches[0]
            print(f"✅ Found raw data at: {raw_path}")
            break
    else:
        print("❌ Could not find dataset. Please update the paths manually.")
        print("   Available input directories:")
        if os.path.exists('/kaggle/input'):
            for item in os.listdir('/kaggle/input'):
                print(f"   - /kaggle/input/{item}")

image_size = (224, 224)

# Fixed Custom ViT-based Segmentation Model
class FixedViTSegmentationModel(nn.Module):
    """
    Fixed Vision Transformer based segmentation model with robust feature handling
    """
    def __init__(self, model_name='vit_base_patch16_224', pretrained=True, num_classes=1, img_size=224):
        super().__init__()
        
        self.img_size = img_size
        
        # Load ViT backbone from timm
        self.vit_backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            img_size=img_size,
            in_chans=3,
            num_classes=0,  # Remove classification head
            global_pool=''  # Remove global pooling
        )
        
        # Determine feature dimensions more robustly
        self._determine_feature_dimensions()
        
        # Create decoder based on determined dimensions
        self._create_decoder(num_classes)
        
        print(f"🏗️  Fixed ViT Segmentation Model initialized")
        print(f"   Encoder: {model_name}")
        print(f"   Feature dim: {self.feature_dim}")
        print(f"   Spatial size: {self.spatial_h}x{self.spatial_w}")
        print(f"   Parameters: {sum(p.numel() for p in self.parameters()):,}")
    
    def _determine_feature_dimensions(self):
        """Robustly determine the output dimensions of the ViT backbone"""
        self.vit_backbone.eval()
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, self.img_size, self.img_size)
            features = self.vit_backbone(dummy_input)
            
            print(f"🔍 ViT output shape: {features.shape}")
            
            if len(features.shape) == 3:  # [B, N, D] - sequence format
                batch_size, seq_len, feature_dim = features.shape
                
                # Check if CLS token is present
                expected_patches = (self.img_size // 16) ** 2  # Assuming patch size 16
                if seq_len == expected_patches + 1:
                    # CLS token present, remove it
                    features = features[:, 1:, :]
                    seq_len = seq_len - 1
                
                self.feature_dim = feature_dim
                self.spatial_h = self.spatial_w = int(np.sqrt(seq_len))
                self.has_cls_token = (seq_len != expected_patches)
                
            elif len(features.shape) == 4:  # [B, D, H, W] - spatial format
                batch_size, feature_dim, h, w = features.shape
                self.feature_dim = feature_dim
                self.spatial_h, self.spatial_w = h, w
                self.has_cls_token = False
                
            else:
                raise ValueError(f"Unexpected ViT output shape: {features.shape}")
    
    def _create_decoder(self, num_classes):
        """Create decoder layers based on spatial dimensions"""
        # Calculate number of upsampling layers needed
        current_size = self.spatial_h
        target_size = self.img_size
        num_upsample_layers = int(np.log2(target_size // current_size))
        
        print(f"   Current spatial size: {current_size}x{current_size}")
        print(f"   Target size: {target_size}x{target_size}")
        print(f"   Upsampling layers needed: {num_upsample_layers}")
        
        decoder_layers = []
        in_channels = self.feature_dim
        
        # Progressive upsampling with feature reduction
        for i in range(num_upsample_layers):
            out_channels = max(64, in_channels // 2)
            
            decoder_layers.extend([
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ])
            
            in_channels = out_channels
        
        # Final refinement layers
        decoder_layers.extend([
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        ])
        
        self.decoder = nn.Sequential(*decoder_layers)
        
        # Final segmentation head
        self.segmentation_head = nn.Sequential(
            nn.Conv2d(32, num_classes, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # Convert grayscale to RGB if needed
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
        
        batch_size = x.shape[0]
        
        # Extract features with ViT backbone
        features = self.vit_backbone(x)
        
        # Handle different output formats
        if len(features.shape) == 3:  # [B, N, D]
            # Remove CLS token if present
            if features.shape[1] == self.spatial_h * self.spatial_w + 1:
                features = features[:, 1:, :]  # Remove CLS token
            
            # Reshape to spatial format [B, D, H, W]
            features = features.transpose(1, 2).reshape(
                batch_size, self.feature_dim, self.spatial_h, self.spatial_w
            )
        
        # Decoder path
        x = self.decoder(features)
        
        # Final segmentation
        output = self.segmentation_head(x)
        
        return output

# Simplified ViT model using proven architecture
class SimpleViTSegmentationModel(nn.Module):
    """
    Simplified ViT segmentation model with fixed architecture
    """
    def __init__(self, pretrained=True, num_classes=1):
        super().__init__()
        
        # Use a well-tested encoder from segmentation_models_pytorch
        try:
            self.model = smp.Unet(
                encoder_name='timm-efficientnet-b4',
                encoder_weights="imagenet" if pretrained else None,
                in_channels=3,
                classes=num_classes,
                activation='sigmoid'
            )
            print("✅ Using EfficientNet-B4 encoder (transformer-like architecture)")
        except:
            # Fallback to ResNet if EfficientNet fails
            self.model = smp.Unet(
                encoder_name='resnet34',
                encoder_weights="imagenet" if pretrained else None,
                in_channels=3,
                classes=num_classes,
                activation='sigmoid'
            )
            print("✅ Using ResNet34 encoder (fallback)")
        
        print(f"   Parameters: {sum(p.numel() for p in self.parameters()):,}")
    
    def forward(self, x):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1)
        return self.model(x)

# Comprehensive evaluation metrics class
class SegmentationMetrics:
    def __init__(self):
        self.metrics_history = []
    
    def calculate_dice_coefficient(self, y_true, y_pred, smooth=1e-5):
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()
        intersection = np.sum(y_true_flat * y_pred_flat)
        dice = (2.0 * intersection + smooth) / (np.sum(y_true_flat) + np.sum(y_pred_flat) + smooth)
        return dice
    
    def calculate_iou(self, y_true, y_pred, smooth=1e-5):
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()
        intersection = np.sum(y_true_flat * y_pred_flat)
        union = np.sum(y_true_flat) + np.sum(y_pred_flat) - intersection
        iou = (intersection + smooth) / (union + smooth)
        return iou
    
    def calculate_sensitivity_specificity(self, y_true, y_pred):
        y_true_flat = y_true.flatten().astype(bool)
        y_pred_flat = y_pred.flatten().astype(bool)
        
        tp = np.sum(y_true_flat & y_pred_flat)
        tn = np.sum(~y_true_flat & ~y_pred_flat)
        fp = np.sum(~y_true_flat & y_pred_flat)
        fn = np.sum(y_true_flat & ~y_pred_flat)
        
        sensitivity = tp / (tp + fn + 1e-5)
        specificity = tn / (tn + fp + 1e-5)
        precision = tp / (tp + fp + 1e-5)
        accuracy = (tp + tn) / (tp + tn + fp + fn + 1e-5)
        f1_score = 2 * (precision * sensitivity) / (precision + sensitivity + 1e-5)
        
        return {
            'sensitivity': sensitivity, 'specificity': specificity,
            'precision': precision, 'accuracy': accuracy, 'f1_score': f1_score,
            'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn
        }
    
    def calculate_hausdorff_distance(self, y_true, y_pred):
        try:
            y_true_binary = (y_true > 0.5).astype(np.uint8)
            y_pred_binary = (y_pred > 0.5).astype(np.uint8)
            
            contours_true, _ = cv2.findContours(y_true_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours_pred, _ = cv2.findContours(y_pred_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if len(contours_true) == 0 or len(contours_pred) == 0:
                return float('inf')
            
            true_points = np.vstack(contours_true).squeeze()
            pred_points = np.vstack(contours_pred).squeeze()
            
            hausdorff_dist = max(
                directed_hausdorff(true_points, pred_points)[0],
                directed_hausdorff(pred_points, true_points)[0]
            )
            
            return hausdorff_dist
        except:
            return float('inf')
    
    def calculate_volume_similarity(self, y_true, y_pred):
        vol_true = np.sum(y_true > 0.5)
        vol_pred = np.sum(y_pred > 0.5)
        
        vol_similarity = 1 - abs(vol_true - vol_pred) / (vol_true + vol_pred + 1e-5)
        relative_vol_error = abs(vol_true - vol_pred) / (vol_true + 1e-5)
        
        return {
            'volume_similarity': vol_similarity,
            'relative_volume_error': relative_vol_error,
            'true_volume': vol_true,
            'pred_volume': vol_pred
        }
    
    def evaluate_batch(self, y_true_batch, y_pred_batch, threshold=0.5):
        batch_metrics = {
            'dice': [], 'iou': [], 'sensitivity': [], 'specificity': [],
            'precision': [], 'accuracy': [], 'f1_score': [], 'hausdorff': [],
            'volume_similarity': [], 'relative_volume_error': []
        }
        
        for i in range(len(y_true_batch)):
            y_true = y_true_batch[i]
            y_pred = (y_pred_batch[i] > threshold).astype(np.float32)
            
            dice = self.calculate_dice_coefficient(y_true, y_pred)
            iou = self.calculate_iou(y_true, y_pred)
            sens_spec = self.calculate_sensitivity_specificity(y_true, y_pred)
            hausdorff = self.calculate_hausdorff_distance(y_true, y_pred)
            vol_metrics = self.calculate_volume_similarity(y_true, y_pred)
            
            batch_metrics['dice'].append(dice)
            batch_metrics['iou'].append(iou)
            batch_metrics['sensitivity'].append(sens_spec['sensitivity'])
            batch_metrics['specificity'].append(sens_spec['specificity'])
            batch_metrics['precision'].append(sens_spec['precision'])
            batch_metrics['accuracy'].append(sens_spec['accuracy'])
            batch_metrics['f1_score'].append(sens_spec['f1_score'])
            batch_metrics['hausdorff'].append(hausdorff)
            batch_metrics['volume_similarity'].append(vol_metrics['volume_similarity'])
            batch_metrics['relative_volume_error'].append(vol_metrics['relative_volume_error'])
        
        summary_metrics = {}
        for key, values in batch_metrics.items():
            valid_values = [v for v in values if not np.isinf(v) and not np.isnan(v)]
            if valid_values:
                summary_metrics[f'{key}_mean'] = np.mean(valid_values)
                summary_metrics[f'{key}_std'] = np.std(valid_values)
            else:
                summary_metrics[f'{key}_mean'] = 0.0
                summary_metrics[f'{key}_std'] = 0.0
        
        return summary_metrics, batch_metrics

# Dataset class for ViT (RGB input)
class FemurDatasetViT(Dataset):
    def __init__(self, image_files, mask_files):
        self.image_files = image_files
        self.mask_files = mask_files
        print(f"📊 ViT Dataset initialized with {len(image_files)} samples")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        try:
            # Load DICOM image
            img_path = self.image_files[idx]
            ds_img = pydicom.dcmread(img_path)
            img = ds_img.pixel_array.astype(np.float32)
            img = cv2.resize(img, image_size)
            
            # Improved normalization
            img_min, img_max = img.min(), img.max()
            if img_max > img_min:
                img = (img - img_min) / (img_max - img_min)
            else:
                img = np.zeros_like(img)
            
            # Convert to RGB for ViT (duplicate grayscale channel)
            img_rgb = np.stack([img, img, img], axis=0)

            # Load DICOM mask
            mask_path = self.mask_files[idx]
            ds_mask = pydicom.dcmread(mask_path)
            mask = ds_mask.pixel_array.astype(np.float32)
            mask = cv2.resize(mask, image_size, interpolation=cv2.INTER_NEAREST)
            mask = (mask > 0).astype(np.float32)  # binarize
            mask = np.expand_dims(mask, axis=0)

            return torch.tensor(img_rgb), torch.tensor(mask)
        
        except Exception as e:
            print(f"❌ Error loading sample {idx}: {e}")
            return torch.zeros(3, *image_size), torch.zeros(1, *image_size)

# File collection function
def collect_file_pairs(raw_root, mask_root, ext=".dcm"):
    print(f"🔍 Scanning for {ext} files...")
    print(f"   Raw path: {raw_root}")
    print(f"   Mask path: {mask_root}")
    
    if not os.path.exists(raw_root):
        print(f"❌ Raw path doesn't exist: {raw_root}")
        return [], []
    
    if not os.path.exists(mask_root):
        print(f"❌ Mask path doesn't exist: {mask_root}")
        return [], []
    
    image_paths, mask_paths = [], []
    cases = sorted(os.listdir(raw_root))
    
    print(f"📁 Found {len(cases)} potential case directories")
    
    for case in tqdm(cases, desc="Processing cases"):
        raw_dir = os.path.join(raw_root, case)
        mask_dir = os.path.join(mask_root, case.replace("-input", "-seg"))
        
        if not os.path.isdir(raw_dir) or not os.path.isdir(mask_dir): 
            continue

        raw_files = sorted([f for f in os.listdir(raw_dir) if f.endswith(ext)])
        mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith(ext)])

        raw_map = {os.path.splitext(f)[0]: os.path.join(raw_dir, f) for f in raw_files}
        mask_map = {os.path.splitext(f)[0]: os.path.join(mask_dir, f) for f in mask_files}

        common = sorted(set(raw_map) & set(mask_map))
        image_paths.extend([raw_map[k] for k in common])
        mask_paths.extend([mask_map[k] for k in common])

    print(f"✅ Found {len(image_paths)} matched pairs")
    return image_paths, mask_paths

# Load model with multiple fallback options
def load_vit_model(use_custom=True):
    print("🔄 Loading ViT model...")
    
    try:
        if use_custom:
            print("   Attempting to load fixed custom ViT model...")
            model = FixedViTSegmentationModel(
                model_name='vit_base_patch16_224',
                pretrained=True,
                num_classes=1
            )
        else:
            print("   Using simplified model...")
            model = SimpleViTSegmentationModel(pretrained=True, num_classes=1)
        
        model = model.to(device)
        model.eval()
        
        # Test model with dummy input
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224).to(device)
            dummy_output = model(dummy_input)
            print(f"✅ Model test successful! Output shape: {dummy_output.shape}")
        
        return model
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        if use_custom:
            print("   Trying simplified model...")
            return load_vit_model(use_custom=False)
        return None

# Comprehensive evaluation function
def evaluate_vit_model(model, test_loader, metrics_calculator, device):
    print("🔍 Performing comprehensive ViT model evaluation...")
    model.eval()
    
    all_predictions = []
    all_ground_truths = []
    inference_times = []
    
    with torch.no_grad():
        for imgs, masks in tqdm(test_loader, desc="Evaluating ViT Model"):
            imgs = imgs.to(device)
            
            start_time = time.time()
            outputs = model(imgs)
            inference_time = time.time() - start_time
            inference_times.append(inference_time / imgs.size(0))
            
            pred_np = outputs.cpu().numpy()
            true_np = masks.numpy()
            
            all_predictions.extend(pred_np)
            all_ground_truths.extend(true_np)
    
    summary_metrics, detailed_metrics = metrics_calculator.evaluate_batch(
        all_ground_truths, all_predictions
    )
    
    summary_metrics['avg_inference_time'] = np.mean(inference_times)
    summary_metrics['std_inference_time'] = np.std(inference_times)
    
    return summary_metrics, detailed_metrics, all_predictions, all_ground_truths, inference_times

# Visualization functions
def print_evaluation_results(summary_metrics):
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE ViT SEGMENTATION EVALUATION RESULTS")
    print("="*80)
    
    print("\n📊 PRIMARY METRICS:")
    print("-" * 50)
    print(f"Dice Similarity Coefficient: {summary_metrics['dice_mean']:.4f} ± {summary_metrics['dice_std']:.4f}")
    print(f"Intersection over Union:     {summary_metrics['iou_mean']:.4f} ± {summary_metrics['iou_std']:.4f}")
    print(f"F1-Score:                    {summary_metrics['f1_score_mean']:.4f} ± {summary_metrics['f1_score_std']:.4f}")
    
    print("\n📈 CLASSIFICATION METRICS:")
    print("-" * 50)
    print(f"Sensitivity (Recall):        {summary_metrics['sensitivity_mean']:.4f} ± {summary_metrics['sensitivity_std']:.4f}")
    print(f"Specificity:                 {summary_metrics['specificity_mean']:.4f} ± {summary_metrics['specificity_std']:.4f}")
    print(f"Precision:                   {summary_metrics['precision_mean']:.4f} ± {summary_metrics['precision_std']:.4f}")
    print(f"Accuracy:                    {summary_metrics['accuracy_mean']:.4f} ± {summary_metrics['accuracy_std']:.4f}")
    
    print("\n⚡ PERFORMANCE METRICS:")
    print("-" * 50)
    print(f"Average Inference Time:      {summary_metrics['avg_inference_time']*1000:.2f} ± {summary_metrics['std_inference_time']*1000:.2f} ms")
    
    dice = summary_metrics['dice_mean']
    if dice > 0.9:
        performance = "Excellent"
    elif dice > 0.8:
        performance = "Good"
    elif dice > 0.7:
        performance = "Fair"
    else:
        performance = "Poor"
    
    print(f"\n🎭 Overall Segmentation Quality: {performance} (DSC = {dice:.4f})")
    print("="*80)

# Main execution
if __name__ == "__main__":
    try:
        metrics_calculator = SegmentationMetrics()
        
        print("📂 Loading dataset...")
        raw_files, mask_files = collect_file_pairs(raw_path, mask_path)
        
        if len(raw_files) == 0:
            print("❌ No data found! Please check your dataset paths.")
        else:
            _, test_images, _, test_masks = train_test_split(
                raw_files, mask_files, test_size=0.3, random_state=42
            )
            
            test_dataset = FemurDatasetViT(test_images, test_masks)
            batch_size = 4 if torch.cuda.is_available() else 2
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
            
            print(f"📊 Test samples: {len(test_dataset)}, Batch size: {batch_size}")
            
            vit_model = load_vit_model(use_custom=True)
            
            if vit_model is not None:
                if len(test_dataset) > 0:
                    print("\n🔬 PERFORMING COMPREHENSIVE ViT MODEL EVALUATION...")
                    
                    summary_metrics, detailed_metrics, all_predictions, all_ground_truths, inference_times = evaluate_vit_model(
                        vit_model, test_loader, metrics_calculator, device
                    )
                    
                    print_evaluation_results(summary_metrics)
                    
                    print(f"\n✅ Evaluation completed!")
                    print(f"   Total samples: {len(test_dataset)}")
                    print(f"   Avg Dice: {summary_metrics['dice_mean']:.4f}")
                    print(f"   Avg IoU: {summary_metrics['iou_mean']:.4f}")
            else:
                print("❌ Failed to load ViT model.")
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        
    print("\n🎉 Script execution completed!")

🚀 Starting Femur Segmentation Evaluation with Fixed ViT Model
✅ GPU Available: Tesla T4
   Memory: 14.7 GB
📂 Loading dataset...
🔍 Scanning for .dcm files...
   Raw path: /kaggle/input/unet-dataset/data/raw
   Mask path: /kaggle/input/unet-dataset/data/mask
📁 Found 8 potential case directories


Processing cases: 100%|██████████| 8/8 [00:00<00:00, 126.35it/s]

✅ Found 4289 matched pairs
📊 ViT Dataset initialized with 1287 samples
📊 Test samples: 1287, Batch size: 4
🔄 Loading ViT model...
   Attempting to load fixed custom ViT model...


🔍 ViT output shape: torch.Size([1, 197, 768])
   Current spatial size: 14x14
   Target size: 224x224
   Upsampling layers needed: 4
🏗️  Fixed ViT Segmentation Model initialized
   Encoder: vit_base_patch16_224
   Feature dim: 768
   Spatial size: 14x14
   Parameters: 92,147,937
✅ Model test successful! Output shape: torch.Size([1, 1, 224, 224])

🔬 PERFORMING COMPREHENSIVE ViT MODEL EVALUATION...
🔍 Performing comprehensive ViT model evaluation...


Evaluating ViT Model: 100%|██████████| 322/322 [00:23<00:00, 13.78it/s]



🎯 COMPREHENSIVE ViT SEGMENTATION EVALUATION RESULTS

📊 PRIMARY METRICS:
--------------------------------------------------
Dice Similarity Coefficient: 0.4639 ± 0.4987
Intersection over Union:     0.4639 ± 0.4987
F1-Score:                    0.0000 ± 0.0000

📈 CLASSIFICATION METRICS:
--------------------------------------------------
Sensitivity (Recall):        0.0000 ± 0.0000
Specificity:                 1.0000 ± 0.0000
Precision:                   0.0000 ± 0.0000
Accuracy:                    0.9953 ± 0.0063

⚡ PERFORMANCE METRICS:
--------------------------------------------------
Average Inference Time:      2.45 ± 1.14 ms

🎭 Overall Segmentation Quality: Poor (DSC = 0.4639)

✅ Evaluation completed!
   Total samples: 1287
   Avg Dice: 0.4639
   Avg IoU: 0.4639

🎉 Script execution completed!
